In [ ]:
# !pip install pandas
import pandas as pd
import numpy as np

# print(pd.__version__)
df = pd.read_csv('./data/Disease_symptom_and_patient_profile_dataset.csv')

# df.shape
array = df['Disease'].to_numpy()
category = np.unique(array)

# print(array.shape, category.shape)# # # # 
# catego

# import numpy as np

# mystr = "100110"
# print np.array(list(mystr))
# # ['1' '0' '0' '1' '1' '0']

# print np.array(list(mystr), dtype=int)
# # [1 0 0 1 1 0]

In [3]:
df.isna().sum

<bound method DataFrame.sum of      Disease  Fever  Cough  Fatigue  Difficulty Breathing    Age  Gender  \
0      False  False  False    False                 False  False   False   
1      False  False  False    False                 False  False   False   
2      False  False  False    False                 False  False   False   
3      False  False  False    False                 False  False   False   
4      False  False  False    False                 False  False   False   
..       ...    ...    ...      ...                   ...    ...     ...   
344    False  False  False    False                 False  False   False   
345    False  False  False    False                 False  False   False   
346    False  False  False    False                 False  False   False   
347    False  False  False    False                 False  False   False   
348    False  False  False    False                 False  False   False   

     Blood Pressure  Cholesterol Level  Outcome Variable

In [4]:
df.duplicated().sum()
print(df)

df = df.drop_duplicates()
# print(df)


         Disease Fever Cough Fatigue Difficulty Breathing  Age  Gender  \
0      Influenza   Yes    No     Yes                  Yes   19  Female   
1    Common Cold    No   Yes     Yes                   No   25  Female   
2         Eczema    No   Yes     Yes                   No   25  Female   
3         Asthma   Yes   Yes      No                  Yes   25    Male   
4         Asthma   Yes   Yes      No                  Yes   25    Male   
..           ...   ...   ...     ...                  ...  ...     ...   
344       Stroke   Yes    No     Yes                   No   80  Female   
345       Stroke   Yes    No     Yes                   No   85    Male   
346       Stroke   Yes    No     Yes                   No   85    Male   
347       Stroke   Yes    No     Yes                   No   90  Female   
348       Stroke   Yes    No     Yes                   No   90  Female   

    Blood Pressure Cholesterol Level Outcome Variable  
0              Low            Normal         Positive  

In [5]:
# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install matplotlib
# !pip install scikit-learn
# !pip install catboost
# !pip install xgboost
# !pip install dill
# !pip install flask

from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(df, test_size = 0.26, random_state = 42)

train_data.to_csv('../artifacts/train.csv', index = False, header = True)
test_data.to_csv('../artifacts/test.csv', index = False, header = True)

# print(train_data)
print("Ingestion of the data is completed")



Ingestion of the data is completed


In [6]:
#Data trasformation
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler

numerical_columns = ["Age"]

categorical_columns = [
                "Fever",
                "Cough",
                "Fatigue",
                "Difficulty Breathing",
                "Gender",
                "Blood Pressure",
                "Cholesterol Level",
                "Outcome Variable",
            ]

# print(f'{train_arr}')


In [7]:
num_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="median")),
                    ("scalar", StandardScaler(with_mean=False))
                ]
            )

cat_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("one_hot_encoder", OneHotEncoder()),
                    ("scaler", StandardScaler(with_mean=False))

                ]
            )


preprocessor = ColumnTransformer(
                [
                    ("num_pipeline", num_pipeline, numerical_columns),
                    ("cat_pipeline", cat_pipeline, categorical_columns)
                ]
            )


In [41]:

train_df = pd.read_csv('../artifacts/train.csv')
test_df = pd.read_csv('../artifacts/test.csv')


target_column_name = "Disease"
numerical_columns = ["Age"]


input_feature_train_df = train_df.drop(columns=[target_column_name], axis=1)
target_feature_train_df = train_df[[target_column_name]]

input_feature_test_df = test_df.drop(columns=[target_column_name], axis=1)
target_feature_test_df = test_df[[target_column_name]]

# print(f"Applying preprocessing object on training dataframe and testing dataframe {input_feature_test_df}")
one = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

output_feature_train_df = one.fit_transform(target_feature_train_df)
output_feature_test_df = one.transform(target_feature_test_df)

output_feature_test_df
# print(output_feature_train_arr)


,Disease_Acne,Disease_Allergic Rhinitis,Disease_Alzheimer's Disease,Disease_Anemia,Disease_Anxiety Disorders,Disease_Appendicitis,Disease_Asthma,Disease_Atherosclerosis,Disease_Autism Spectrum Disorder (ASD),Disease_Bipolar Disorder,...,Disease_Testicular Cancer,Disease_Tetanus,Disease_Thyroid Cancer,Disease_Tourette Syndrome,Disease_Tuberculosis,Disease_Turner Syndrome,Disease_Ulcerative Colitis,Disease_Urinary Tract Infection,Disease_Urinary Tract Infection (UTI),Disease_Williams Syndrome
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:

import numpy as np

input_feature_train_arr = preprocessor.fit_transform(input_feature_train_df)
input_feature_test_arr = preprocessor.transform(input_feature_test_df)

print(input_feature_train_arr)

[[2.76195032 0.         2.00816597 ... 2.00816597 0.         2.00129986]
 [5.52390063 2.00816597 0.         ... 2.00816597 2.00129986 0.        ]
 [3.55107898 0.         2.00816597 ... 0.         0.         2.00129986]
 ...
 [3.15651465 2.00816597 0.         ... 2.00816597 2.00129986 0.        ]
 [5.1293363  0.         2.00816597 ... 2.00816597 2.00129986 0.        ]
 [3.15651465 2.00816597 0.         ... 2.00816597 2.00129986 0.        ]]


In [28]:
train_arr = np.c_[input_feature_train_arr, np.array(output_feature_train_df)]
test_arr = np.c_[input_feature_test_arr, np.array(output_feature_test_df)]

print(train_arr[1])
print("Saved preprocesssing object.")

[5.52390063 2.00816597 0.         2.01610014 0.         0.
 2.25174829 2.36116775 0.         0.         2.00008117 0.
 0.         2.00129986 0.         0.         2.00816597 2.00129986
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         0.         0.         0. 

In [11]:
# save object
import dill

with open('../artifacts/preprocessor.pkl', "wb") as file_obj:
        dill.dump(preprocessor, file_obj)


In [29]:
from sklearn.ensemble import (
    AdaBoostRegressor, 
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


X_train,y_train,X_test,y_test = (
    input_feature_train_arr,
    np.array(output_feature_train_df),
    input_feature_test_arr,
    np.array(output_feature_test_df)
)

print(train_arr[1])
print("Saved preprocesssing object.")
# print(X_train.shape ,y_train.shape ,X_test.shape ,y_test.shape)
# print(type(test_arr))
# ValueError: X has 19 features, but LinearRegression is expecting 37 features as input.

[5.52390063 2.00816597 0.         2.01610014 0.         0.
 2.25174829 2.36116775 0.         0.         2.00008117 0.
 0.         2.00129986 0.         0.         2.00816597 2.00129986
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         0.         0.         0. 

In [34]:
def evaluate_models(X_train, y_train, X_test, y_test, models):
    report = {}

    for i in range(len(list(models))):
        model = list(models.values())[i]

        model.fit(X_train,y_train)

        y_train_pred = model.predict(X_train)

        y_test_pred = model.predict(X_test)

        train_model_score = r2_score(y_train, y_train_pred)

        test_model_score = r2_score(y_test, y_test_pred)

        report[list(models.keys())[i]] = test_model_score

    return report

In [81]:

from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVC

models = {
        "Random Forest" : RandomForestRegressor(),
        "Decision Tree" : DecisionTreeRegressor(),
        # "Gradient Boosting" : GradientBoostingRegressor(),
        "Linear Regression" : LinearRegression(),
        "K-Neighbors Regressor" : KNeighborsRegressor(),
        "XGBRegressor" : XGBRegressor(),
        # "CatBoosting Regressor" : CatBoostRegressor(verbose = False),
        # "AdaBoost Regressor" : AdaBoostRegressor()
        # "SVM" : SVC(kernel="rbf", gamma=0.5, C=1.0)
}

 

model_report:dict = evaluate_models(
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    models = models
)
# ValueError: y should be a 1d array, got an array of shape (222, 101) instead.
# print(X_train)

In [4]:

best_model_score = max(sorted(model_report.values()))

best_model_name = list(model_report.keys())[
    list(model_report.values()).index(best_model_score)
]

best_model = models[best_model_name]

if best_model_score < 0.6:
    print("No best model found")

print("Best found model on both training and testing dataset")

# save object
import dill

with open('../artifacts/best_model.pkl', "wb") as file_obj:
        dill.dump(best_model, file_obj)


predicted = best_model.predict(X_test)

r2_sco = r2_score(y_test, predicted)

print(r2_sco)

NameError: name 'model_report' is not defined

In [3]:
print("shivam")

shivam
